# Lastfm top interactive dashboard

## Ricardo Vargas (September 2023)

### Import libraries

In [1]:
import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display

### Load the CSV file

In [2]:
data = pd.read_csv("reduce_data.csv")

### Extract year and month

In [3]:
# Convert the 'utc_time' column to datetime format
data['utc_time'] = pd.to_datetime(data['utc_time'], format='%d %b %Y, %H:%M')

# Extract year and month from the 'utc_time' column
data['year'] = data['utc_time'].dt.year
data['month'] = data['utc_time'].dt.month

### Make the selection widget

In [4]:
# Widgets for selecting year and month range
start_year = widgets.IntSlider(min=data['year'].min(), max=data['year'].max(), value=data['year'].min(), description='Start Year')
end_year = widgets.IntSlider(min=data['year'].min(), max=data['year'].max(), value=data['year'].max(), description='End Year')
start_month = widgets.IntSlider(min=1, max=12, value=1, description='Start Month')
end_month = widgets.IntSlider(min=1, max=12, value=12, description='End Month')

### Filtering data and graphs 

In [5]:
# Display function for the charts
def display_charts(start_y, end_y, start_m, end_m):
    # Filter data
    filtered_data = data[(data['year'] >= start_y) & (data['year'] <= end_y) & 
                         (data['month'] >= start_m) & (data['month'] <= end_m)]
    
    # Top Artists Chart
    top_artists = filtered_data['artist'].value_counts().head(20).reset_index()
    top_artists.columns = ['Artist', 'Count']
    fig1 = px.bar(top_artists, x='Artist', y='Count', title='Top 20 Most Listened Artists')
    fig1.show()
    
    # Top Albums Chart
    top_albums = filtered_data.groupby(['artist', 'album']).size().reset_index(name='Count')
    top_albums = top_albums.sort_values(by='Count', ascending=False).head(20)

    # Define the explicit order for the x-axis based on the album names in descending order of count
    album_order = top_albums['album'].tolist()

    fig2 = px.bar(top_albums, x='album', y='Count', color='artist',
                  title='Top 20 Most Listened Albums', labels={'album': 'Album', 'Count': 'Count'},
                  category_orders={"album": album_order})
    fig2.show()


    # Top Tracks Chart
    top_tracks = filtered_data.groupby(['artist', 'album', 'track']).size().reset_index(name='Count')
    top_tracks = top_tracks.sort_values(by='Count', ascending=False)
    top_tracks = top_tracks.head(20)

    # Define the explicit order for the x-axis based on the track names in descending order of count
    track_order = top_tracks['track'].tolist()

    fig3 = px.bar(top_tracks, x='track', y='Count', color='artist',
                  title='Top 20 Most Listened Tracks', labels={'track': 'Track', 'Count': 'Count'},
                  category_orders={"track": track_order})
    fig3.show()



    

### Interact widgets

In [6]:
widgets.interact(display_charts,
                 start_y=start_year,
                 end_y=end_year,
                 start_m=start_month,
                 end_m=end_month)

interactive(children=(IntSlider(value=2007, description='Start Year', max=2023, min=2007), IntSlider(value=202…

<function __main__.display_charts(start_y, end_y, start_m, end_m)>